In [1]:
import pandas as pd
import numpy as np


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [2]:
from io import StringIO

col = ['author', 'text']
train = train[pd.notnull(train['text'])]


train['author_id'] = train['author'].factorize()[0]
author_id_df = train[['author', 'author_id']].drop_duplicates().sort_values('author_id')
author_to_id = dict(author_id_df.values)
id_to_category = dict(author_id_df[['author_id', 'author']].values)
train.head()

,id,text,author,author_id
0,id26305,"This process, however, afforded me no means of...",EAP,0
1,id17569,It never once occurred to me that the fumbling...,HPL,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=3, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(train.text).toarray()
labels = train.author_id
features.shape

(19579, 14561)

In [4]:
from sklearn.feature_selection import chi2
import numpy as np

N = 
for author, author_id in sorted(author_to_id.items()):
  features_chi2 = chi2(features, labels == author_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(author))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'EAP':
  . Most correlated unigrams:
. perdita
. raymond
  . Most correlated bigrams:
. chess player
. said dupin
# 'HPL':
  . Most correlated unigrams:
. gilman
. old
  . Most correlated bigrams:
. shunned house
. new england
# 'MWS':
  . Most correlated unigrams:
. perdita
. raymond
  . Most correlated bigrams:
. fellow creatures
. lord raymond


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(train['text'], train['author'], random_state = 3, test_size = 0.1, train_size = 0.9)
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(X_train).toarray()
labels = []
for y in y_train:
    labels.append(author_to_id[y])

mnb = MultinomialNB().fit(features, y_train)


#lr =  LogisticRegression(random_state=0, solver = "liblinear").fit(features, y_train)

#lsv = LinearSVC().fit(features, y_train)

#rdc = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0).fit(features, y_train)

In [32]:
correct = 0
wrong = 0
predicitions = mnb.predict(tfidf.transform(X_test))
y_sorted = []
for y in y_test:
    y_sorted.append(y)
    
for i in range(len(predicitions)):
    if(predicitions[i] == y_sorted[i]):
        correct += 1
    else:
        wrong += 1
    

In [33]:
print(correct)
print(wrong)
print(correct/(wrong+correct))

1611
347
0.8227783452502554


/home/jake/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jake/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/jake/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jake/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/jake/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarn